### Observable trends for a fictional free game with MTX
  * Male players far outnumber female players with around 80% of the total. However, both genders on average spend about the same amount on MTX, with male players spending slightly more.
  
  
  * 20 - 24 year-olds make up the majority of players at 40% with 15 - 19 year-olds in second with 24%. Both of the age groups spend about the same on average, with the 20 - 24 year-olds spending slightly more. 40+ year-olds make up the smallest group with only 3%.
  
  
  * *Betrayal, Whisper of Grieving Widows* and *Arcane Gem* are the most purchased items but *Retribution Axe* with its higher price comes in at the most profitable. *Retribution Axe* is the only item to be in the top 5 lists of most popular and most profitable. 

In [1]:
#dependencies
import pandas as pd
import numpy as np

In [2]:
#filepath
purch_path = "Resources/purchase_data.json"

#read into a df
purch_df = pd.read_json(purch_path)

In [3]:
'''Total Players'''

#get total # of players by getting the length of the SN column
player_count = len(purch_df["SN"].unique())

#make a dataframe with Total Players as the column title
player_count_df = pd.DataFrame({"Total Players": [player_count]})

player_count_df

,Total Players
0,573


In [4]:
#make a function that formats to 2 decimal places and/or adds $ sign
def formatter (df, column, *args):
    if 2 in args:
        x = df[column].map("{:,.2f}".format)
    else:
        x = df[column].map("${:,.2f}".format)
    return x

In [5]:
'''Purchasing Analysis (Total)'''

#find the # of unique items
item_count = len(purch_df["Item Name"].unique())

#find the average purchase price
avg_price = purch_df["Price"].mean()

#find the total # of purchases
purch_count = purch_df["Price"].count()

#find the total revenue
rev_total = purch_df["Price"].sum()

#make a dictionary, which will be used to make the df
tot_purch_anls_dict = {"Number of Unique Items": item_count, "Average Price": avg_price, 
                       "Number of Purchases": purch_count, "Total Revenue": rev_total}
#make the df
purch_anls_tot_df = pd.DataFrame(tot_purch_anls_dict, index = [0])
purch_anls_tot_df

#format the df using the money_format function
purch_anls_tot_df["Average Price"] = formatter(purch_anls_tot_df, "Average Price")
purch_anls_tot_df["Total Revenue"] = formatter(purch_anls_tot_df, "Total Revenue")


#put the columns in the proper place
purch_anls_tot_df = purch_anls_tot_df[[ 'Number of Unique Items', 'Average Price', 'Number of Purchases', 'Total Revenue']]

purch_anls_tot_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$2.93,780,"$2,286.33"


In [6]:
'''Gender Demographics'''

#make a df for unique people
unique_people_df = purch_df[["Gender", "Age", "SN"]].drop_duplicates(subset="SN")

#find total count of genders
gend_tot = unique_people_df["Gender"].count()

#find # of males and percentage
male_count = unique_people_df["Gender"].value_counts()['Male']
male_perc = (male_count / gend_tot) * 100

#find # of females and percentage
female_count = unique_people_df["Gender"].value_counts()['Female']
female_perc = (female_count / gend_tot) * 100

#find the # of other/non-disclosed and percentage
na_gend_count = gend_tot - male_count - female_count
na_gend_perc = (na_gend_count / gend_tot) * 100

#make a df for gender demographics
gend_demo_df = pd.DataFrame({
    "Gender": ["Male", "Female", "Other / Non-Disclosed"],
    "Percentage of Players": [male_perc, female_perc, na_gend_perc],
    "Total Count": [male_count, female_count, na_gend_count]})

#make gender the index
gend_demo_df = gend_demo_df.set_index("Gender")

#put the columns in the proper place
gend_demo_df = gend_demo_df[["Percentage of Players", "Total Count"]]

#remove Gender title
gend_demo_df.index.name = None

#format data
gend_demo_df["Percentage of Players"] = formatter(gend_demo_df, "Percentage of Players", 2)
gend_demo_df

,Percentage of Players,Total Count
Male,81.15,465
Female,17.45,100
Other / Non-Disclosed,1.40,8


In [7]:
'''Purchasing Analysis (Gender)'''

#group by gender and assign to a df
gender_grpd = purch_df.groupby(["Gender"])

#find the purchase count
gen_purch_count = gender_grpd["Price"].count()

#find the average purch price
gen_avg_price = gender_grpd["Price"].mean()

#find the total of the purchases
gen_total_purch = gender_grpd["Price"].sum()

#find the normalized totals
gen_norm_totals = gen_total_purch / gend_demo_df["Total Count"]

#make the df
gen_purch_anls_df = pd.DataFrame({
    "Purchase Count": gen_purch_count, 
    "Average Purchase Price": gen_avg_price, 
    "Total Purchase Value": gen_total_purch,
    "Normalized Totals": gen_norm_totals})

#put the columns in the proper place
gen_purch_anls_df = gen_purch_anls_df[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]

#format the df
gen_purch_anls_df["Average Purchase Price"] = formatter(gen_purch_anls_df, "Average Purchase Price")
gen_purch_anls_df["Total Purchase Value"] = formatter(gen_purch_anls_df, "Total Purchase Value")
gen_purch_anls_df["Normalized Totals"] = formatter(gen_purch_anls_df, "Normalized Totals")

gen_purch_anls_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,136,$2.82,$382.91,$3.83
Male,633,$2.95,"$1,867.68",$4.02
Other / Non-Disclosed,11,$3.25,$35.74,$4.47


In [8]:
'''Age Demographics'''

#create bins
bins = [0, 10, 15, 20, 25, 30, 35, 40, 120]

#create the bin labels
bin_labels = [">10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#copy unique_people_df into a new df
up_bin_df = unique_people_df.copy(deep=True)

#put ages into the bins 
up_bin_df["Age Range"] = pd.cut(up_bin_df["Age"], bins, labels=bin_labels)

#group the df by the Age Range
up_age_grpd = up_bin_df.groupby("Age Range")

#find the total count
age_count = up_age_grpd["SN"].count()

#find the percentage of players
age_perc = (age_count / player_count) * 100

#make the df for age demographics
age_demo_df = pd.DataFrame({"Percentage of Players": age_perc, "Total Count": age_count})

#format the df
age_demo_df["Percentage of Players"] = formatter(age_demo_df, "Percentage of Players", 2)

#remove index title
age_demo_df.index.name = None

age_demo_df

,Percentage of Players,Total Count
>10,3.84,22
10-14,9.42,54
15-19,24.26,139
20-24,40.84,234
25-29,9.08,52
30-34,7.68,44
35-39,4.36,25
40+,0.52,3


In [9]:
'''Purchasing Analysis (Age)'''

#copy purch_df into a new df
purch_bin_df = purch_df.copy(deep=True)

#put ages into the bins
purch_bin_df["Age Range"] = pd.cut(purch_bin_df["Age"], bins, labels=bin_labels)

#group the df by the Age Range
purch_age_grpd = purch_bin_df.groupby("Age Range")

#find the purchase count
age_purch_count = purch_age_grpd["Price"].count()

#find the avg purch price
age_avg_price = purch_age_grpd["Price"].mean()

#find the total purchase value
age_total_purch = purch_age_grpd["Price"].sum()

#find the normalized totals
age_norm_totals = age_total_purch / age_demo_df["Total Count"]

#make the df
age_purch_anls_df = pd.DataFrame({
    "Purchase Count": age_purch_count, 
    "Average Purchase Price": age_avg_price, 
    "Total Purchase Value": age_total_purch,
    "Normalized Totals": age_norm_totals})

#remove index title
age_purch_anls_df.index.name = None

#put the columns in the proper place
age_purch_anls_df = age_purch_anls_df[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]]

#format the df
age_purch_anls_df["Average Purchase Price"] = formatter(age_purch_anls_df, "Average Purchase Price")
age_purch_anls_df["Total Purchase Value"] = formatter(age_purch_anls_df, "Total Purchase Value")
age_purch_anls_df["Normalized Totals"] = formatter(age_purch_anls_df, "Normalized Totals")

age_purch_anls_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
>10,32,$3.02,$96.62,$4.39
10-14,78,$2.87,$224.15,$4.15
15-19,184,$2.87,$528.74,$3.80
20-24,305,$2.96,$902.61,$3.86
25-29,76,$2.89,$219.82,$4.23
30-34,58,$3.07,$178.26,$4.05
35-39,44,$2.90,$127.49,$5.10
40+,3,$2.88,$8.64,$2.88


In [10]:
'''Top Spenders'''

#group the df by SN
sn_grpd = purch_df.groupby("SN")

#find total purch value
sn_total_purch = sn_grpd["Price"].sum()

#find purch count
sn_purch_count = sn_grpd["Price"].count()

#find the average purchase price
sn_avg_price = sn_grpd["Price"].mean()

#make a df
top_spenders_df = pd.DataFrame({
    "Purchase Count": sn_purch_count,
    "Average Purchase Price": sn_avg_price,
    "Total Purchase Value": sn_total_purch})

#get the top 5 spenders by total purch value
top_spenders_df = top_spenders_df.nlargest(5, "Purchase Count")

#put the columns in the proper place
top_spenders_df = top_spenders_df[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]

#format the df
top_spenders_df["Average Purchase Price"] = formatter(top_spenders_df, "Average Purchase Price")
top_spenders_df["Total Purchase Value"] = formatter(top_spenders_df, "Total Purchase Value")

#show df with the total purch value sorted
top_spenders_df.sort_values("Total Purchase Value", ascending=True)


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Mindimnya67,4,$3.18,$12.74
Saedue76,4,$3.39,$13.56
Undirrala66,5,$3.41,$17.06
Hailaphos89,4,$1.47,$5.87
Qarwen67,4,$2.49,$9.97


In [11]:
'''Most Popular Items'''

#group the df by item id and item name 
id_grpd = purch_df.groupby(["Item ID", "Item Name"] )

#find the purchase count
id_purch_count = id_grpd["Price"].count()

#find the total purch value
id_total_purch = id_grpd["Price"].sum()

#find the item price
id_item_price = id_total_purch / id_purch_count

#make the df 
items_grpd_df = pd.DataFrame({
    "Purchase Count": id_purch_count,
    "Item Price": id_item_price,
    "Total Purchase Value": id_total_purch})

#put the columns in the proper place
items_grpd_df = items_grpd_df[["Purchase Count", "Item Price", "Total Purchase Value"]]

#copy the df
pop_items_df = items_grpd_df.copy(deep=True)

#format the df
pop_items_df["Item Price"] = formatter(pop_items_df, "Item Price")
pop_items_df["Total Purchase Value"] = formatter(pop_items_df, "Total Purchase Value")

#get the 5 most popular items by purch count
pop_items_df.nlargest(5, "Purchase Count")


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
13,Serenity,9,$1.49,$13.41
31,Trickster,9,$2.07,$18.63
34,Retribution Axe,9,$4.14,$37.26


In [12]:
'''Most Profitable Items'''

#copy the df
most_prof_df = items_grpd_df.copy(deep=True)

#get the 5 most profitable items by total purchase value
most_prof_df = most_prof_df.nlargest(5, "Total Purchase Value")

#format the df
most_prof_df["Item Price"] = formatter(most_prof_df, "Item Price")
most_prof_df["Total Purchase Value"] = formatter(most_prof_df, "Total Purchase Value")

most_prof_df

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88
